In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)



Mounted at /content/drive


In [ ]:
!python -m spacy download fr_core_news_sm

     |████████████████████████████████| 14.7 MB 5.1 MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-py3-none-any.whl size=14727026 sha256=1d9788fceed78ad358b689bba59b47a62b4396ca7d1145684d652981cd5946da
  Stored in directory: /tmp/pip-ephem-wheel-cache-rop4j4lj/wheels/c9/a6/ea/0778337c34660027ee67ef3a91fb9d3600b76777a912ea1c24
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:

import spacy
nlp = spacy.load('fr_core_news_sm')

In [ ]:
chemincorpus = "/content/drive/MyDrive/Stage Defense/Corpus/"
cheminw2v = chemincorpus + 'W2V_mentions_Defense.csv'
cheminbow = chemincorpus + 'BoW_mentions_Defense.csv'
cheminfeatures = chemincorpus +"features_mentions_Defense.csv"


Fonctions utiles

In [ ]:
def equilibre_classes(X,y):
  '''Retire aléatoirement des exemples des classes surnnuméraires, à la fois dans X et dans y
  Renvoie X et y avec toutes les classes ayant le meme nombre d'exemple que la classe minoritaire'''
  dicoclasses = {k:v for k,v in zip(*np.unique(y, return_counts=True))}
  print("avant",dicoclasses)
  mini = min(dicoclasses.values())
  while any((v >mini for v in dicoclasses.values())):
    
    for cat in dicoclasses:
      if dicoclasses[cat] > mini:
        nb_hasard = np.random.choice(np.where(y==cat)[0],1)
        X = np.delete(X,nb_hasard, axis=0)
        y = np.delete(y,nb_hasard, axis=0)
        dicoclasses[cat]-=1

  print("apres",dicoclasses)
  return X,y
  #for i in range(len(X)):


In [ ]:
def aff_res(cm,model):
    '''
    cm est la matrice de confusion
    classes est la liste des classes, obtenue par model.classes_ (où model est le modèle de classification généré)
    '''

    #%matplotlib inline
    get_ipython().run_line_magic('matplotlib', 'inline')
    
    class_names = model.classes_

    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    # create heatmap
    sns.heatmap(pd.DataFrame(cm), annot=True, cmap="YlGnBu" ,fmt='g')
    
    #set label names
    ax.xaxis.set_label_position("top")
    ax.xaxis.set_ticklabels(class_names)
    ax.yaxis.set_ticklabels(class_names)
    
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

Chargement des données

In [ ]:
dfbow = pd.read_csv(cheminbow, sep=",",index_col=0)
dfw2v = pd.read_csv(cheminw2v, sep=",",index_col=0)
dffeatures = pd.read_csv(cheminfeatures, sep=",",index_col=0)

In [ ]:
chemindata = chemincorpus+"Toutes_mentions_Defense_annotees.csv"

In [ ]:
dfannot = pd.read_csv(chemindata,sep=",", index_col=0, keep_default_na=False,na_values=["NA"])
annotations = dfannot.loc[:,:"positionnement_contexte"]
annotations = annotations.fillna("NA")

In [ ]:
annotations.mention[annotations.mention.str.contains("froid")]

109        des tours de bureaux froides et impersonnelles
1391                          d'un quartier froid et vide
1496    comme un quartier uniquement dédié au travail,...
Name: mention, dtype: object

In [ ]:
dfbow.index

Int64Index([   1,    2,    3,    4,    5,    6,    9,   10,   11,   12,
            ...
            1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462],
           dtype='int64', length=827)

In [ ]:
annotations = annotations.loc[dfbow.index,:]

In [ ]:
X_bow_mention = dfbow.iloc[:,:159]
X_bow_contexte = dfbow.iloc[:,160:]

In [ ]:
X_w2v_mention = dfw2v.iloc[:,:500]
X_w2v_contexte = dfw2v.iloc[:,501:]

# Predictions mention

In [ ]:
def validation_croisee(X,y,clf):
  y_pred = cross_val_predict(clf,X,y=y)
  report = classification_report(y,y_pred)
  print(report)

In [ ]:
def split_labels(y):
  newy = []
  for labels in y:
    labelsplit = [label.strip() for label in labels.split("/")]
    if labelsplit == ['']:
      print(labels)
    newy.append(labelsplit)
  return newy

## Procédé_nommage

### Extraction de la Norme

In [ ]:
def separe_norme(dffeatures):
  index_normes = []
  for i in dffeatures.index:
    if dffeatures.loc[i,"is_norme"]:
      index_normes.append(i)
      #print(y.iloc[i,0])
  return index_normes


In [ ]:
index_normes = separe_norme(dffeatures)

In [ ]:
df_pred = pd.DataFrame(index=annotations.index)
df_pred["procédé_de_nommage"] = "autre"
df_pred[df_pred.index.isin(index_normes)]="norme"

In [ ]:
index_not_norme = annotations[~annotations.index.isin(index_normes)].index

In [ ]:
y_norme_autre = annotations.loc[:,"procédé_de_nommage"].copy()
y_norme_autre.loc[index_not_norme] = "autre"

In [ ]:
print(classification_report(df_pred.procédé_de_nommage,y_norme_autre))

              precision    recall  f1-score   support

       autre       1.00      1.00      1.00       439
       norme       1.00      1.00      1.00       388

    accuracy                           1.00       827
   macro avg       1.00      1.00      1.00       827
weighted avg       1.00      1.00      1.00       827



### Autres procédés de nommage

In [ ]:
from sklearn.utils import shuffle

Préparation données

#### Descripteurs Linguistiques

Symbolique

In [ ]:
def classifie_procédés(X):
  y_pred_symbo = []
  for ordi,super,lieu,dist_lieu,geo,dist_geo,dist_norme in  X:
    labels = [0,0,0,0]
    if ordi or super:
      labels[0] = 1
    if lieu :
      labels[3] = 1
    if geo : 
      labels[2] = 1
    if all([dist < 0.50 for dist in {dist_lieu,dist_geo,dist_norme}]):
      labels[1] = 1
    y_pred_symbo.append(labels)
  return y_pred_symbo

In [ ]:
y_pred_symbo = classifie_procédés(X_procédé)
report = classification_report(y_procédé,y_pred_symbo,target_names=MB.classes_)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report)

                        precision    recall  f1-score   support

            classement       1.00      0.96      0.98        23
             métaphore       0.85      0.85      0.85        20
référence géographique       0.99      0.97      0.98        68
             type lieu       0.93      0.96      0.95        57

             micro avg       0.95      0.95      0.95       168
             macro avg       0.94      0.94      0.94       168
          weighted avg       0.95      0.95      0.95       168
           samples avg       0.95      0.95      0.94       168



## Cible_mention

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import random

In [ ]:
def reequilibre_classes(X,y,cible_support):
  
  classes,counts = np.unique(np.array([label  for labels in y for label in labels]),return_counts=True)
  dico_counts = {classe: count for classe,count in zip(classes,counts)}
  dico_counts = dict(sorted(dico_counts.items(),key=lambda x:x[1] , reverse=True))
  print("avant", dico_counts)
  #print(X)

  new_X,new_y = [], []
  if not isinstance(X,list):
    X= X.tolist()
  nb_sup,nb_add = 0,0
  nb_total = len(X)
  for label in dico_counts:
    print(label,"avant", dico_counts)
    while dico_counts[label]  < cible_support   or dico_counts[label]  > cible_support:
      i = random.randint(0,len(y)-1)
      labels = y[i]
      if label not in labels : continue
      if dico_counts[label] < cible_support :
        for lab in labels:
          dico_counts[lab] +=1
        X.append(X[i])
        y.append(y[i])
        nb_add +=1
      elif dico_counts[label] > cible_support :
        if any((dico_counts[lab] < cible_support-5 for lab in labels) ): continue
        for lab in labels:
          dico_counts[lab] -=1
        del X[i]
        del y[i]
        nb_sup+=1
    
    print(label,"après", dico_counts)

  classes,counts = np.unique(np.array([label  for labels in y for label in labels]),return_counts=True)
  dico_counts = {classe: count for classe,count in zip(classes,counts)}
  print(f"exemples avant : {nb_total}", f"ajoutés : {nb_add}", f"supprimés : {nb_sup}", f"exemples après : {len(X)}", sep = " | ")
  return X,y
      



  new_dico_counts =   {classe: count for classe,count in zip(*np.unique(np.array([label  for labels in new_y for label in labels]),return_counts=True))}
  print("a la fin",new_dico_counts)

Préparation des données

In [ ]:
X_cible_m = X_w2v_mention.loc[index_not_norme,:].to_numpy()
y_cible_m = split_labels(annotations.loc[index_not_norme,:].cible_mention)
X_cible_m, y_cible_m =reequilibre_classes(X_cible_m,y_cible_m,15)
MB = MultiLabelBinarizer()
y_cible_m = MB.fit_transform(y_cible_m)
X_cible_m,y_cible_m = shuffle(X_cible_m,y_cible_m)


avant {'économie': 336, 'urbanisme': 59, 'localisation': 55, 'architecture': 19, 'espace de vie': 14, 'taille': 8, 'changement': 6, 'disparité': 4}
économie avant {'économie': 336, 'urbanisme': 59, 'localisation': 55, 'architecture': 19, 'espace de vie': 14, 'taille': 8, 'changement': 6, 'disparité': 4}
économie après {'économie': 15, 'urbanisme': 57, 'localisation': 21, 'architecture': 17, 'espace de vie': 12, 'taille': 8, 'changement': 6, 'disparité': 4}
urbanisme avant {'économie': 15, 'urbanisme': 57, 'localisation': 21, 'architecture': 17, 'espace de vie': 12, 'taille': 8, 'changement': 6, 'disparité': 4}
urbanisme après {'économie': 15, 'urbanisme': 15, 'localisation': 20, 'architecture': 17, 'espace de vie': 10, 'taille': 8, 'changement': 6, 'disparité': 4}
localisation avant {'économie': 15, 'urbanisme': 15, 'localisation': 20, 'architecture': 17, 'espace de vie': 10, 'taille': 8, 'changement': 6, 'disparité': 4}
localisation après {'économie': 15, 'urbanisme': 15, 'localisatio

Apprentissage

In [ ]:
MB.classes_

array(['architecture', 'changement', 'disparité', 'espace de vie',
       'localisation', 'taille', 'urbanisme', 'économie'], dtype=object)

In [ ]:
#validation_croisee(X_w2v_mention.loc[index_not_norme,:].to_numpy(),annotations.loc[index_not_norme,:].procédé_de_nommage,LogisticRegression())
clf = MLPClassifier(hidden_layer_sizes=100,max_iter=700)


In [ ]:
y_pred = cross_val_predict(clf,X=X_cible_m,y=y_cible_m)


In [ ]:
report = classification_report(y_cible_m,y_pred,target_names=MB.classes_)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report)

               precision    recall  f1-score   support

 architecture       1.00      0.81      0.89        21
   changement       0.88      0.93      0.90        15
    disparité       1.00      0.93      0.97        15
espace de vie       0.93      0.62      0.74        21
 localisation       0.94      0.79      0.86        19
       taille       0.88      0.93      0.90        15
    urbanisme       0.72      0.87      0.79        15
     économie       1.00      1.00      1.00        26

    micro avg       0.92      0.86      0.89       147
    macro avg       0.92      0.86      0.88       147
 weighted avg       0.93      0.86      0.88       147
  samples avg       0.84      0.83      0.83       147



In [ ]:
import pickle

In [ ]:
path_model_cible_mention = '/content/drive/MyDrive/Stage Defense/Scripts/Modeles/cible_mention_model.sav'

In [ ]:
pickle.dump(clf, open(path_model_cible_mention, 'wb'))

## Polarité

Regression Logistique supervisé

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# stopwords de nltk
nltk_stopwords = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import gensim
from sklearn.metrics.pairwise import cosine_similarity 

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
chemin = "/content/drive/MyDrive/Stage Defense"
cheminmodeleW2V = chemin+"/Ressources/frWac_no_postag_no_phrase_500_cbow_cut100.bin"
model_frWac = gensim.models.KeyedVectors.load_word2vec_format(cheminmodeleW2V, binary=True, unicode_errors="ignore")

df_pola = pd.read_csv(chemin+"/Ressources/polarité_mots.csv",sep="\t",names=["mot","POS","polarité","confiance"])


In [ ]:
mots_positifs = df_pola[(df_pola.polarité == "POS")&(df_pola.confiance!='0,33%')].mot.tolist()
mots_negatifs = df_pola[(df_pola.polarité == "NEG")&(df_pola.confiance!='0,33%')].mot.tolist()
mots_neutres = df_pola[(df_pola.polarité == "NEUTRE")&(df_pola.confiance!='0,33%')].mot.tolist()

In [ ]:
def cos_sim(a,b):
  return cosine_similarity(a.reshape(1, -1),b.reshape(1, -1))[0][0]

In [ ]:
def moyenne_vecteurs(liste_mots):
  liste_vecteurs = []
  for mot in liste_mots:
    if mot in model_frWac.vocab:
      liste_vecteurs.append(model_frWac[mot])
  return np.mean(np.array(liste_vecteurs),axis=0)

In [ ]:
vecteur_neutre = moyenne_vecteurs(mots_neutres)
vecteur_positif = moyenne_vecteurs(mots_positifs)
vecteur_negatif = moyenne_vecteurs(mots_negatifs)

In [ ]:
def detecte_polarité(X,y,df):
  y_pred = []
  sims = []
  for i, vecteur_mention in enumerate(X):
    negativite = cos_sim(vecteur_mention,vecteur_negatif)
    positivite = cos_sim(vecteur_mention,vecteur_positif)
    neutralite = cos_sim(vecteur_mention,vecteur_neutre)
    #print(df.iloc[i], negativite,neutralite,positivite, sep="\t")
    choix_pola = {positivite:"positif",negativite:"négatif",neutralite:"neutre"}
    #pola = choix_pola[max([positivite,negativite,neutralite])]
    max_pola = max([positivite,negativite,neutralite])
    if max_pola == positivite:
      pola="positif"
    elif max_pola == negativite:
      pola="négatif"
    else:
      pola = "neutre"
    y_pred.append(pola)
    if y[i] == "positif":
      print(negativite,neutralite,positivite)
    #print(df.iloc[i], negativite,neutralite,positivite,max([positivite,negativite,neutralite]),choix_pola,pola,y[i], sep="\t")
    sims.append([negativite,positivite,neutralite])
  return y_pred,sims

In [ ]:
def detecte_polarité_sims(X,y):
  y_pred = []
  sims = []
  for i, mention in enumerate(X):
    doc = nlp(mention)
    pola = "neutre"
    negs,poss,neuts = [], [] , []
    for token in doc:
      if token.text == "seul": continue

      if token.pos_ not in ["ADV","NOUN","PROPN","ADJ","INTJ","VERB"] or token.lemma_ not in model_frWac.vocab or token.lemma_ in nltk_stopwords|{"déjà"}: continue
      vecteur_token = model_frWac[token.lemma_]
      negativite = cos_sim(vecteur_token,vecteur_negatif)
      positivite = cos_sim(vecteur_token,vecteur_positif)
      neutralite = cos_sim(vecteur_token,vecteur_neutre)
      #print(df.iloc[i], negativite,neutralite,positivite, sep="\t")
      choix_pola = {positivite:"positif",negativite:"négatif",neutralite:"neutre"}
      #pola = choix_pola[max([positivite,negativite,neutralite])]
      max_pola = max([positivite,negativite,neutralite])
      negs.append(negativite)
      poss.append(positivite)
      neuts.append(negativite)
      if y[i] == "négatif":
        pass
        #print(choix_pola, token.lemma_, mention)
      if max_pola == positivite and positivite > 0.1:
        
        pola="positif"
        if y[i]!="positif":
          print(token, token.lemma_,positivite, neutralite, negativite, y[i], mention)
        break
      elif max_pola == negativite and  negativite > 0.1 :
        pola="négatif"
        
        #print(token, token.lemma_,positivite, neutralite, negativite, y[i], mention)
        break
      
    y_pred.append(pola)
    neg = max(negs) if negs else 0
    neut = max(neuts) if neuts else 0
    pos = max(poss) if poss else 0
    sims.append([neg, neut,pos])
    if y[i] == "négatif":
      pass

  return y_pred,sims

In [ ]:
mentions_pola = annotations.loc[index_not_norme,"mention"]

In [ ]:
y_pred,sims = detecte_polarité_sims(mentions_pola,y_pola_m)
print(classification_report(y_pola_m,y_pred))

tours tour 0.2474078 0.1430674 0.0927223 neutre des tours
tours tour 0.2474078 0.1430674 0.0927223 négatif des tours de bureaux froides et impersonnelles
tours tour 0.2474078 0.1430674 0.0927223 neutre les tours
tours tour 0.2474078 0.1430674 0.0927223 neutre des tours
tours tour 0.2474078 0.1430674 0.0927223 neutre des tours
génération génération 0.19152321 0.028488718 0.0015842337 neutre La Défense nouvelle génération
tours tour 0.2474078 0.1430674 0.0927223 neutre les 70 tours et les sièges sociaux
tours tour 0.2474078 0.1430674 0.0927223 neutre aux tours
tours tour 0.2474078 0.1430674 0.0927223 neutre des tours
tours tour 0.2474078 0.1430674 0.0927223 neutre au pied des tours
grâce grâce 0.3998186 0.17416021 0.26655558 neutre grâce au quartier d'affaires de la défense
capitale capitale 0.1351002 0.08169999 0.04624751 neutre le quartier d'affaires de la capitale
tours tour 0.2474078 0.1430674 0.0927223 neutre des tours
              precision    recall  f1-score   support

      neu

## Positionnement mention

In [ ]:
X_posit_m = dffeatures.loc[index_not_norme,"contient_guillemet"].to_numpy()
y_posit_m = annotations.loc[index_not_norme,:].positionnement_mention.to_numpy()

In [ ]:
y_pred_posit_m =  []
for v in X_posit_m :
  if v:
    print(v)
    y_pred_posit_m.append("cité")
  else:
    y_pred_posit_m.append("pris en charge")



1
1
1
1


In [ ]:
report = classification_report(y_posit_m,y_pred_posit_m)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report)

                precision    recall  f1-score   support

      attribué       0.00      0.00      0.00         1
          cité       0.50      0.15      0.24        13
pris en charge       0.97      1.00      0.98       425

      accuracy                           0.97       439
     macro avg       0.49      0.38      0.41       439
  weighted avg       0.96      0.97      0.96       439



Conclusion : Impossible à déterminer sans le contexte